# Imports

In [1]:
import sys
import os
sys.path.append('/Users/fauberma/MPG/Scripts/droplet-phenotyping')

In [2]:
from Tools.leica_tools import RawLoader
from Tools.sample_tools import Sample
from Tools.db_tools import DbManager


# Data prep

In [3]:
expID = 'NKIP_FA_058'
rawloader = RawLoader(expID)
rawloader.frame_df

,droplet_size,size_range,image_index,t_index,time,condition,path
frameID,,,,,,,
0,83,5,0,0,2,Positive,/Volumes/T7/DMI/NKIP_FA_058/NKIP_FA_058.lif
1,83,5,1,0,2,Negative,/Volumes/T7/DMI/NKIP_FA_058/NKIP_FA_058.lif
2,83,5,2,0,2,Unsorted,/Volumes/T7/DMI/NKIP_FA_058/NKIP_FA_058.lif
3,83,5,3,0,3,Positive,/Volumes/T7/DMI/NKIP_FA_058/NKIP_FA_058.lif
4,83,5,4,0,3,Negative,/Volumes/T7/DMI/NKIP_FA_058/NKIP_FA_058.lif
5,83,5,5,0,3,Unsorted,/Volumes/T7/DMI/NKIP_FA_058/NKIP_FA_058.lif
6,83,5,3,1,4,Positive,/Volumes/T7/DMI/NKIP_FA_058/NKIP_FA_058.lif
7,83,5,4,1,4,Negative,/Volumes/T7/DMI/NKIP_FA_058/NKIP_FA_058.lif
8,83,5,5,1,4,Unsorted,/Volumes/T7/DMI/NKIP_FA_058/NKIP_FA_058.lif


# Droplet detection

Execute a preview run of the droplet detection. An image will be saved to the exp folder in the analyses directory. 
If droplets are not well detected consider changing the droplet size estimate in setup.xlsx (re-run RawLoader API).

In [4]:
frameID = 0
sample = Sample(expID, frameID)
sample.detect_droplets(mode='sweep')
sample.visualize_droplets(channel=0,save=True)

5789 droplets in frame 0 detected 


Run droplet detection through all frames of the experiment. drop_register.csv will be created at the end of the process.

In [ ]:
dbm = DbManager()
dbm.generate_drop_register(expID, mode='sweep')
dbm.generate_tfrecord(expID)

In [ ]:
dbm.generate_tfrecord(expID)

# Outlier detection

In [5]:
expID = 'NKIP_FA_058'
dbm = DbManager()
dbm.detect_outliers(expID, model_name='outlier_v2.h5')

3385/3385 ━━━━━━━━━━━━━━━━━━━━ 119s 35ms/step


/Users/fauberma/mambaforge/envs/dropcnn/lib/python3.12/contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)
2024-07-15 14:45:01.838647: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-07-15 14:45:20.156029: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-07-15 14:45:20.984383: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-07-15 14:45:21.142451: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-07-15 14:45:21.297907: W tensorflow/core/framewor

In [ ]:
sample = Sample(expID, 0)
sample.reload_droplets()
sample.visualize_droplets(channel=0)

# WP Generation

In [ ]:
dbm = DbManager()
dbm.generate_wp('NKIP_FA_058', exclude_query='outlier == True')

# Inference

In [4]:
expID = 'NKIP_FA_055'
dbm = DbManager()

In [5]:
dbm.cell_count(expID, model_name='cell_count_v2.h5')

2024-07-18 14:36:39.943030: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


4538/4538 ━━━━━━━━━━━━━━━━━━━━ 474s 104ms/step


2024-07-18 14:44:33.960091: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
/Users/fauberma/mambaforge/envs/dropcnn/lib/python3.12/contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)
